In [1]:
import pandas as pd
import tensorflow as tf
import os
os.chdir('/Users/zhangle/tensorflow-without-a-phd/tensorflow-mnist-tutorial')
import tensorflowvisu

def batchnorm(Ylogits, is_test, iteration, offset, convolutional=False):
    # adding the iteration prevents from averaging across non-existing iterations
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, iteration) 
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_averages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_averages

def no_batchnorm(Ylogits, is_test, iteration, offset, convolutional=False):
    return Ylogits, tf.no_op()

In [43]:
d = pd.read_stata('/Users/zhangle/Desktop/demo1.dta')
d.head(2)

# Normalization
def Normalization(x):
    return [(float(i)-min(x))/float(max(x)-min(x)) for i in x]
# 归一化
import numpy as np
for i in ["ages","sbp","dbp","bmi","tchol","hdl","glucosef","duration1"]:
    d.ix[:,i]=Normalization(np.array(d.ix[:,i]))
d.describe()

X = d.ix[:,'ages':'duration1']
Y = d.ix[:,'ep1_chdmi']

In [44]:
d.describe()

,ages,sbp,dbp,bmi,tchol,hdl,glucosef,duration1
count,2000.000000,2000.000000,2000.000000,1996.000000,2000.000000,1997.000000,1905.000000,2000.000000
mean,0.493845,0.371054,0.413667,0.270594,0.467650,0.313336,0.132905,0.638785
std,0.148981,0.155093,0.146433,0.130686,0.150091,0.118098,0.074370,0.268002
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.379982,0.258333,0.305556,0.184916,0.348271,0.227964,0.097762,0.361440
50%,0.482143,0.366667,0.416667,0.247905,0.482714,0.291793,0.117786,0.777318
75%,0.597478,0.458333,0.513889,0.332472,0.582586,0.379939,0.146643,0.878205
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
from sklearn import model_selection
test_size = 0.15
train_X, train_Y, test_X, test_Y = model_selection.train_test_split\
(X,Y,test_size=test_size,random_state=42)

In [8]:
# neural network with 4 layers
#
#       · · · ·                (input data, flattened pixels)          X [batch, 10] 
# \x/x\x/x\x/x\x/x\x/ ✞      -- fully connected layer (relu+dropout)   W1 [10, 108]      B1[108]
#  · · · · · · · · ·                                                   Y1 [batch, 108]
#     \x/x\x/x\x/ ✞          -- fully connected layer (relu+dropout)   W2 [108, 10]      B2[10]
#       · · · ·                                                        Y2 [batch, 10]
#         \x/ ✞              -- fully connected layer (softmax)        W3 [10, 2]        B3[2]
#          ·                                                           Y3 [batch, 2]
X = tf.placeholder("float",[None,10])
Y_ = tf.placeholder("float",[None,2])
lr = tf.placeholder(tf.float32) # variable learning rate
pkeep = tf.placeholder(tf.float32) # probability of keeping a node during dropout =1.0 at test,0.75 at training
step = tf.placeholder(tf.int32) # step for variable learning rate
tst = tf.placeholder(tf.bool)# train/test selector for batch normalisation
iter = tf.placeholder(tf.int32)# training iteration

# Three layers and their numbers of neurons(the last layer has 2 softmax neurons)
L = 108
M = 10
# Weights initialised with small random values between -0.2 and +0.2, 
#Biases are initialised with small positive values( positive: because we are using Relu)
W1 = tf.Variable(tf.truncated_normal([10,L],stddev=0.1))
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M],stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
W3 = tf.Variable(tf.truncated_normal([M, 2],stddev=0.1))
B3 = tf.Variable(tf.ones([2]))

# The model, with dropout, batch normalization at each layer
# batch norm scaling is not useful with relus
# batch norm offsets are used instead of biases
Y1l = tf.matmul(X, W1)
Y1bn, update_ema1 = batchnorm(Y1l, tst, iter, B1)
Y1r = tf.nn.relu(Y1bn)
Y1 = tf.nn.dropout(Y1r, pkeep)

Y2l = tf.matmul(Y1, W2)
Y2bn, update_ema2 = batchnorm(Y2l,tst, iter, B2)
Y2r = tf.nn.relu(Y2bn)
Y2 = tf.nn.dropout(Y2r, pkeep)

Ylogits = tf.matmul(Y2, W3) + B3
Y = tf.nn.softmax(Ylogits)

update_ema = tf.group(update_ema1, update_ema2)

In [17]:
# cross-entropy loss function = -sum(Y_i*log(Yi)), normalised for batches of 100 images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability problems 
# with log(0) which is NaN
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

# accuracy of the trained model, between 0 and 1
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

import math
# the learning rate is: # 0.0001 + 0.03 * (1/e)^(step/10000) i.e. exponential decay from 0.03->0.0001
lr = 0.0001 + tf.train.exponential_decay(0.03, iter, 1000, 1/math.e)
# training step
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

In [15]:
# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(tf.initialize_all_variables())

In [20]:
for i in range(20000):
    batch_X, batch_Y = next_batch(100)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={X: batch_X, Y_: batch_Y, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={X: batch_X, Y_: batch_Y, keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


NameError: name 'next_batch' is not defined

In [ ]:
import os
os.chdir("/Users/zhangle/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets")
import mnist as m

In [ ]:
# matplotlib visualisation
allweights = tf.concat([tf.reshape(W1, [-1]), tf.reshape(W2, [-1]), tf.reshape(W3, [-1])], 0)
allbiases = tf.concat([tf.reshape(B1, [-1]), tf.reshape(B2, [-1]), tf.reshape(B3, [-1])], 0)
# ti use for RELU
allactivations = tf.concat([tf.reduce_max(Y1, [0]), tf.reduce_max(Y2, [0])], 0)
alllogits = tf.concat([tf.reshape(Y1l, [-1]), tf.reshape(Y2l, [-1])], 0)
# 有问题
I = tensorflowvisu.tf_format_mnist_images(X, Y, Y_)
It = tensorflowvisu.tf_format_mnist_images(X, Y, Y_, 1000, lines=25)
datavis = tensorflowvisu.MnistDataVis(title4="Logits", title5="Max activations across batch", histogram4colornum=2, histogram5colornum=2)

In [ ]:
def training_step(i, update_test_data, update_train_data):
    # training on batches of 100 images with 100 labels
    batch_X, batch_Y = next_batch(100)
    
    # compute training values for visualisation
    if update_train_data:
        a, c, im, al, ac, l = sess.run([accuracy, cross_entropy, I, alllogits, allactivations, lr],
                                       feed_dict={X: batch_X, Y_: batch_Y, iter: i, tst: False})
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c) + " (lr:" + str(l) + ")")
        datavis.append_training_curves_data(i, a, c)
        datavis.update_image1(im)
        datavis.append_data_histograms(i, al, ac)

    # compute test values for visualisation
    if update_test_data:
        a, c, im = sess.run([accuracy, cross_entropy, It], {X: mnist.test.images, Y_: mnist.test.labels, tst: True})
        print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ********* test accuracy:" + str(a) + " test loss: " + str(c))
        datavis.append_test_curves_data(i, a, c)
        datavis.update_image2(im)

In [ ]:
# the backpropagation training step
sess.run([train_step, update_ema], feed_dict={X: batch_X, Y_: batch_Y, tst: False, iter: i})

datavis.animate(training_step, iterations=10000+1, train_data_update_freq=20, test_data_update_freq=100, more_tests_at_start=True)

# to save the animation as a movie, add save_movie=True as an argument to datavis.animate
# to disable the visualisation use the following line instead of the datavis.animate line
# for i in range(10000+1): training_step(i, i % 100 == 0, i % 20 == 0)

print("max test accuracy: " + str(datavis.get_max_test_accuracy()))

In [ ]:
sess.run([train_step, update_ema], feed_dict={X: batch_X, Y_: batch_Y, tst: False, iter: i})